In [1]:

x = [   [0,0], [0,1], [0,1] ,
     [1,0], [1,1], [1,2]
]

y = [1,0,1,1,0,1]

col_name = ["gender", "age"]

In [4]:
from collections import Counter



def gini(y):
    ttl = float(len(y))
    cnt = Counter(y)
    result = 1
    for key in cnt:
        result -=  (cnt[key]/ttl)**2
    return result

def DT(x,y, indices, modalities, metric="gini"):
    ''' 
    
    Args:
        (x,y) 数据集
        indices(iterables) :- 目前叶子上的数据点的指标
        modalities(iterables) :- [[modality]],每种特征的取值空间
        metric(str) :- 不纯度度量
    '''

    col_num = len(x[0])
    num_samples = float(len(indices))
    
    x_sample = [x[index] for index in indices]
    y_sample = [y[index] for index in indices]
    
    
    if metric == "gini":
        impurity = gini(y_sample)
        
        impurity_min = 1
        
        leaves = []
        
        for col in range(col_num):
            impurity_A = 0
            leaves_candidate = []
            
            for modality in modalities[col]:
                
                sample_k = [ (i,value) for i,value in enumerate(y_sample) if x_sample[i][col]==modality ]
                y_k = [ele[1] for ele in sample_k]
                if len(y_k) > 0: # 避免生成空叶子
                    gini_k = gini(y_k)
                    impurity_A += len(y_k)/num_samples * gini_k
                
                    leaves_candidate.append( { "sample" : [  indices[ele[0]] for ele in sample_k], "modality" : modality, "impurity" : gini_k}  )
                    
            if impurity_A < impurity_min:
                impurity_min =  impurity_A
                A = col_name[col]
                leaves = leaves_candidate
        
        impurity_reduction = impurity- impurity_min
    return impurity_reduction, A, leaves
        
        

In [22]:
def stop_cond(leaves):
    ''' 叶子结点都为单一类型
    '''
    for leaf in leaves:
        if leaf["impurity"] > 0 :
            return False
    
    return True


def DT_total(x,y):
    # 保证x和y数量一致
    try :
        assert len(x) == len(y)
    except AssertionError:
        raise Exception("The length of x and y must be identical")
        
    
    col_num = len(x[0])
    
    modalities =[ set() for _ in range(col_num)]
    
    # 获取全部特征的取值空间以及确保x没有缺失值
    try:
        for x_i in x:
            assert len(x_i) == col_num
            for col in range(col_num):
                modalities[col].add(x_i[col])
    except AssertionError:
        raise Exception("The length of x must be equal")
        
    ttl = len(x) # 样本大小
    leaves = [] # 叶子结点
    non_leaves = [] # 非叶子结点,每个非叶子结点都有key来表示孩子结点
    
    root = {
        "impurity" : gini(y),
        "sample" : [i for i in range(ttl)],
        "name" : 'v0'
    }
    i = 0

    leaves.append(root)
    
    while not stop_cond(leaves): # 不满足停止条件时
        leaf2split = dict()
        childrenGenerated = []
        attrChosen = ''
        
        impu_reduc_max = 0
        for leaf in leaves:
            if leaf["impurity"] > 0:
                samples = leaf['sample']

                impu_reduc, attr, children = DT(x, y, indices = samples, modalities = modalities)
                impu_reduc_tmp = impu_reduc * len(samples)
                if impu_reduc_tmp > impu_reduc_max:
                    leaf2split = leaf
                    childrenGenerated = filter(None,children)
                    attrChosen = attr

        if not leaf2split: # 若没有可以降低不纯度的split方法,则停止split
            break
        leaves.remove(leaf2split)

        
        leaf2split["children"] = []
        for child in childrenGenerated:
            i += 1
            name = "v{}".format(i)
            child["name"] = name
            leaf2split["children"].append(name)
            leaf2split["attr"] = attrChosen
            
        leaves.extend(childrenGenerated)
        non_leaves.append(leaf2split)
        
         
    return leaves, non_leaves
    

In [23]:
DT_total(x,y)

([{'impurity': 0.0, 'modality': 0, 'name': 'v1', 'sample': [0, 3]},
  {'impurity': 0.0, 'modality': 2, 'name': 'v3', 'sample': [5]},
  {'impurity': 0.5, 'modality': 0, 'name': 'v4', 'sample': [1, 2]},
  {'impurity': 0.0, 'modality': 1, 'name': 'v5', 'sample': [4]}],
 [{'attr': 'age',
   'children': ['v1', 'v2', 'v3'],
   'impurity': 0.4444444444444444,
   'name': 'v0',
   'sample': [0, 1, 2, 3, 4, 5]},
  {'attr': 'gender',
   'children': ['v4', 'v5'],
   'impurity': 0.4444444444444445,
   'modality': 1,
   'name': 'v2',
   'sample': [1, 2, 4]}])

In [ ]:
x